In [ ]:
#016
sum([int(x) for x in str(2**1000)])

In [ ]:
#015
def fact(x):
    if x<=1:
        return x
    else:
        return fact(x-1)*x
print(fact(40)//fact(20)**2)

In [ ]:
#071
import fractions
N = 1000000
m = 1
num = set()
v0 = fractions.Fraction(3,7)
for i in range(N,0,-1):
    j = i*3//7
    v1 = fractions.Fraction(j,i)
    if v1 != v0:
        if v0 - v1 < m:
            m = v0-v1
            mv = v1
print(mv)


In [ ]:
#077

def primes(x):
    sieve = [True for i in range(x)]
    for i in range(2,x):
        if sieve[i]:
            for j in range(2*i,x,i):
                sieve[j] = False
    p = []
    for i in range(2,x):
        if sieve[i]:
            p.append(i)
    return p

numsumcache = dict()


def numsum(x,maxp):
    if (x,maxp) in numsumcache:
        return numsumcache[(x,maxp)]
    count = 0
    for i in p:
        if i <= maxp:
            if i<x:
                count += numsum(x-i,min(i,maxp))
            elif i==x:
                #print(x,maxp,seq+[i])
                count += 1
    numsumcache[(x,maxp)] = count
    return count

p = primes(5000)
i = 2
while numsum(i,i) < 5100:
    print(i,numsum(i,i))
    i+=1

In [ ]:
#056
m = 0
for a in range(100):
    for b in range(100):
        d = [int(c) for c in str(a**b)]
        e = sum(d)
        if m < e:
            m=e
            ma = a
            mb = b
print(ma,mb,m)

In [ ]:
#095
#divisors
import itertools
import time

divcache = dict()
divcache[1] = set([1])
p = set()

def divisors(x):
    if x in divcache:
        return divcache[x]
    sqx = int(x**0.5)+1
    for pn in range(2,sqx):
        if x % pn == 0:
            s = set()
            for j in divisors(x//pn):
                for k in divisors(pn):
                    s.add(j*k)
            divcache[x] = s
            return s
    #print("{0} is a prime number.".format(x),p)
    p.add(x)
    divcache[x] = set([1,x])
    return (1,x)

#make divisors list
#1000: 0.007 sec  
#10000: 0.092 sec
#100000: 1.370 sec
#1000000: 24.719 sec

now = time.time()
N = 1000000
for i in range(1,N+1):
    divisors(i)
print("{0:.3f} sec".format(time.time() - now))

chain = dict()
#make admissible chain
for i in range(1,N+1):
    s = sum(divisors(i)) - i
    chain[i] = s

inspected = set([0])
for i in range(1,N+1):
    if i in inspected:
        continue
    n = i
    path = dict()
    while n not in (0,1):
        if n in inspected:
            if n in path:
                #extract the loop
                cycle = []
                while n not in cycle:
                    cycle.append(n)
                    n = chain[n]
                print(cycle,len(cycle))
            break
        inspected.add(n)
        path[n] = chain[n]
        n = chain[n]
        if n > N:
            break


In [116]:
#096
#sudoku solver




groups = []
nodeg = dict()
for gx in range(3):
    for gy in range(3):
        grids = []
        for px in range(3):
            for py in range(3):
                grids.append((gx*3+px, gy*3+py))
        groups.append(frozenset(grids))
for row in range(9):
    grids = []
    for x in range(9):
        grids.append((x,row))
    groups.append(frozenset(grids))
for column in range(9):
    grids = []
    for y in range(9):
        grids.append((column,y))
    groups.append(frozenset(grids))
for group in groups:
    for node in group:
        nodeg[node] = set()
for group in groups:
    for node in group:
        nodeg[node].add(group)



def drawmap(map,zoom):
    print("#---------+---------+---------#---------+---------+---------#---------+---------+---------#")
    for y in range(9):
        line = "|"
        for x in range(9):
            for s in range(1,10):
                if s in map[x][y]:
                    line += ("%1s"%s)
                else:
                    line += " "
            line += "|"
        print(line)
        if y%3 == 2:
            print("#---------+---------+---------#---------+---------+---------#---------+---------+---------#")


def statmap(map):
    m = 0
    for x in range(9):
        for y in range(9):
            if m < len(map[x][y]):
                m = len(map[x][y])
    return m

    

def reduceset(x,y,valueset,map,queue):
    orig = len(map[x][y])
    map[x][y] -= valueset
    if len(map[x][y]) == 1 and orig > 1:
        memb = list(map[x][y])
        queue.append(("force",(x,y),memb[0]))
    return len(map[x][y])
        


def setvalue(x,y,valueset,map,queue):
    err = False
    for group in nodeg[(x,y)]:
        for nx,ny in group:
            if (nx,ny) != (x,y):
                if 0 == reduceset(nx,ny,valueset,map,queue):
                    err = True
    map[x][y] = valueset
    return err

#solitary number can be fixed.
def solitary(grids,map,queue):
    count = [0] * 10
    for x,y in grids:
        if len(map[x][y]) != 1:
            for memb in map[x][y]:
                count[memb] += 1
    nums = []
    for i in range(10):
        if count[i] == 1:
            nums.append(i)
    for num in nums:
        for x,y in grids:
            if num in map[x][y]:
                queue.append(("force",(x,y),num))


def process(queue,map,mode):
    global groups
    err = False
    if mode == 0:
        if len(queue) > 0:
            command = queue.pop(0)
            if command[0] == "set":
                x,y = command[1]
                value = set((command[2],))
                if len(map[x][y]) > 1:
                    err = setvalue(x,y,value,map,queue)            
            elif command[0] == "force":
                x,y = command[1]
                value = set((command[2],))
                err = setvalue(x,y,value,map,queue)
        return err
    #solitary number in a group. Required for level 3
    if mode > 0:
        while True:
            for group in groups:
                solitary(group,map,queue)
            if len(queue) == 0:
                break
            else:
                while len(queue)>0:
                    if process(queue,map,0):
                        err = True
        return err


def setup():
    global cursor,map,zoom,queue,mode
    cursor = None
    map = [[set([1,2,3,4,5,6,7,8,9]) for i in range(9)] for j in range(9)]
    zoom = 60 #size of one square
    queue = []
    mode = 0
    queue.insert(0,("set",(0,0),8))
    queue.insert(0,("set",(1,2),7))
    queue.insert(0,("set",(1,3),5))
    queue.insert(0,("set",(1,8),9))
    queue.insert(0,("set",(2,1),3))
    queue.insert(0,("set",(2,6),1))
    queue.insert(0,("set",(2,7),8))
    queue.insert(0,("set",(3,1),6))
    queue.insert(0,("set",(3,5),1))
    queue.insert(0,("set",(3,7),5))
    queue.insert(0,("set",(4,2),9))
    queue.insert(0,("set",(4,4),4))
    queue.insert(0,("set",(5,3),7))
    queue.insert(0,("set",(5,4),5))
    queue.insert(0,("set",(6,2),2))
    queue.insert(0,("set",(6,4),7))
    queue.insert(0,("set",(6,8),4))
    queue.insert(0,("set",(7,5),3))
    queue.insert(0,("set",(7,6),6))
    queue.insert(0,("set",(7,7),1))
    queue.insert(0,("set",(8,6),8))

    
    
def setup_for_PE(file):
    global cursor,map,zoom,queue,mode
    cursor = None
    map = [[set([1,2,3,4,5,6,7,8,9]) for i in range(9)] for j in range(9)]
    zoom = 60 #size of one square
    queue = []
    mode = 0
    line = file.readline()
    for i in range(9):
        line = file.readline()
        for j in range(9):
            ch = line[j]
            v  = int(ch)
            if v != 0:
                queue.insert(0,("set",(j,i),v))



def assume(map):
    queue = []
    for x in range(9):
        for y in range(9):
            if len(map[x][y]) > 1:
                nsolution = 0
                nfail = 0
                for v in map[x][y]:
                    print("assumption ({0},{1})=={2}:".format(x,y,v))
                    mapcopy = [[set(map[j][i]) for i in range(9)] for j in range(9)]
                    queue.append(("set",(x,y),v))
                    err = False
                    while len(queue)>0:
                        if process(queue,mapcopy,mode):
                            err = True
                    if not err:
                        status = statmap(mapcopy)
                        if status > 1:
                            ns,nf = assume(mapcopy)
                            nsolution += ns
                            nfail += nf
                        else:
                            #drawmap(mapcopy,zoom)
                            nsolution += 1
                    else:
                        print("failed.")
                        nfail += 1
                return nsolution,nfail
    return 1,0

#setup()
file = open("096sudoku.txt")
for i in range(50):
    setup_for_PE(file)

    mode = 1
    while len(queue)>0:
        process(queue,map,mode)
    #drawmap(map,zoom)

    #assumptions
    #save the map
    assume(map)
    nsol, nfail = assume(map)
    drawmap(map,zoom)
    print(map[0][0],map[1][0],map[2][0])

    #print(list(map[0][0])[0],list(map[1][0])[0],list(map[2][0])[0])

#---------+---------+---------#---------+---------+---------#---------+---------+---------#
|   4     |       8 |  3      |        9| 2       |1        |     6   |    5    |      7  |
|        9|     6   |      7  |  3      |   4     |    5    |       8 | 2       |1        |
| 2       |    5    |1        |       8 |      7  |     6   |   4     |        9|  3      |
#---------+---------+---------#---------+---------+---------#---------+---------+---------#
|    5    |   4     |       8 |1        |  3      | 2       |        9|      7  |     6   |
|      7  | 2       |        9|    5    |     6   |   4     |1        |  3      |       8 |
|1        |  3      |     6   |      7  |        9|       8 | 2       |   4     |    5    |
#---------+---------+---------#---------+---------+---------#---------+---------+---------#
|  3      |      7  | 2       |     6   |       8 |        9|    5    |1        |   4     |
|       8 |1        |   4     | 2       |    5    |  3      |      7  |     6   

In [135]:
#100
#Too slow to solve the real problem.

import fractions
for i in range(10**4,10**5):
    j = int(i/2**0.5)+1
    f = fractions.Fraction((i*(i-1)),(j*(j-1)))
    if f==2:
        print(i)

    
    

23661
